<a href="https://colab.research.google.com/github/kittisaktaoma/colab_practice/blob/master/tmp/caox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [97]:
#! pip install imbalanced-learn

In [99]:
import imblearn 
import pandas as pd

In [52]:
#! git clone https://github.com/Superzchen/iFeature
! python iFeature/iFeature.py --file drive/My\ Drive/sample_test.fasta --type AAC --out AAC.txt

Descriptor type: AAC


In [ ]:
df = pd.read_csv("AAC.txt",sep="\t")
df2 = df.rename({'#': 'id'}, axis='columns')
df_02 = pd.DataFrame(df2.id.str.split('|',1).tolist(),columns = ['A','B'])
df_03 = pd.DataFrame(df_02.B.str.split('|',1).tolist(),columns = ['id','B'])
df_c = pd.concat([df_03.iloc[:,0], df.iloc[:,1:21]], axis=1)
df_c

In [100]:
index_df = pd.read_csv("https://raw.githubusercontent.com/kittisaktaoma/colab_practice/master/tmp/sample.txt",sep="\t")
cdf = pd.merge(df_c,index_df,on='id')
cdf['cat'].value_counts()


0    143
1     16
Name: cat, dtype: int64